## Chapter 10 시퀀스 해킹, 해시, 슬라이스
Vector2d를 능가하는 다차원 Vector 버전을 만들어 본다.

### 10.2 Vector 버전 #1: Vector2d 호환
생성자 시그니처가 달라진 점 외에 요소 두 개를 가진 Vector 클래스는 Vector2d로 수행했던 모든 테스트와 동일한 결과가 나오도록 만들어졌다.

In [1]:
""" [예제 10-2] Vector 최초 버전 """

from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'
    
    def __init__(self, components): # 초기화 함수가 없으면 object.__init()를 호출한다.
        self._components = array(self.typecode, components) # 백터 요소를 배열로 저장한다.
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components) # [0.0, 1.0, 2.0, 3.0, ...] 형태로 출력된다.
                                               # reprlib.repr(list(self._components))로도 가능하지만 단순한 출력을 위해 모든 항목을 list에 복사하는 것은 낭비이다.
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # memoryview를 바로 생성자에 전달한다. 
    
    def __len__(self):
        print("run len")
        return len(self._components)

    def __getitem__(self, index):
        print("run getitem")
        cls = type(self) # 객체의 클래스(Vector)를 가져온다.
        if isinstance(index, slice):
            print("run slice root in getitem")
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral): 
            print("run numbers.Integral root in getitem")
            return self._components[index]
        else:
            print("run other root in getitem")
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
            
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shorcut_names:
                error = 'read only attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value) # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 함수를 호출한다.
        
    def __eq_(self, other):
        if len(self) != len(other):
            return False
        for a, b in zip(self, other):
            if a != b:
                return False
        return True                

In [2]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [3]:
Vector((3.1, 4.2, 5.3))

Vector([3.1, 4.2, 5.3])

In [4]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

### 10.3 프로토콜과 덕 타이핑
객체지향 프로그래밍에서 프로토콜은 문서에만 정의되어 있고 실제 코드에서는 정의되지 않는 비공식 인터페이스다. 예를 들어 파이썬의 시퀀스 프로토콜은 \_\_len\_\_( )과 \_\_getitem\_\_( ) 메서드를 동반할 뿐이다.

In [5]:
""" [예제 10-3] 시퀀스 프로토콜 """
import collections 

Card = collections.namedtuple('card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit)
                       for suit in self.suits
                       for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

### 10.4 Vector 버전 #2: 슬라이스 가능한 시퀀스

아래 예제는 Vector의 슬라이스가 배열을 반환하는 문제점이 있다. (이전의 FrenchDeck도 마찬가지이다.) 
```
class Vector:
    # 중략
    # ...
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]
```

In [6]:
v1 = Vector([3, 4, 5])
len(v1)

run len


3

In [7]:
v1[0], v1[-1]

run getitem
run numbers.Integral root in getitem
run getitem
run numbers.Integral root in getitem


(3.0, 5.0)

In [8]:
v7 = Vector(range(7))
v7[1:4]

run getitem
run slice root in getitem


Vector([1.0, 2.0, 3.0])

#### 10.4.1 슬라이싱의 작동 방식


In [9]:
""" [예제 10-4] __getitem__()과 slice()의 동작 확인 """

class MySeq:
    def __getitem__(self, index):
        return index # 전달받은 인수를 그대로 반환한다.
    
s = MySeq()
print("{!r}".format(s[1]))
print("{!r}".format(s[1:4]))
print("{!r}".format(s[1:4:2]))
print("{!r}".format(s[1:4:2, 9]))   # [] 안에 콤마가 들어가면 __getitem__()이 튜플을 받는다
print("{!r}".format(s[1:4:2, 7:9])) # 튜플 안에 여러 슬라이스 객체가 들어 있을 수 있다.

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [10]:
""" [예제 10-5] slice 클래스의 속성 조사 """
print("{!r}".format(slice)) # slice는 내장된 자료형이다.
print("{!r}".format(dir(slice))) # slice 객체를 조사하면 start, stop, step 속성과 indices() 를 볼 수 있다. 

<class 'slice'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']


In [11]:
help(slice.indices)

Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



#### 10.4.2 슬라이스를 인식하는 __getitem__( )
[예제 10-6]은 Vector가 시퀀스로 동작하기 위해 필요한 \_\_len\_\_( )과 \_\_getitem\_\_( )를 보여준다. 이제는 \_\_getitem\_\_( )이 슬라이싱도 제대로 처리하도록 구현되어 있다.

참고: isintance( )를 많이 사용한다는 것은 객체지향 설계가 잘못되었다는 것을 나타낼 수 있지만, \_\_getitem\_\_( )에서 슬라이스를 처리하는 경우에는 정당화될 수 있다. [예제 10-6]에서 추상 베이스 클래스(ABC)인 numbers.Integral에 대해 테스트하는 것을 보라. isinstance( ) 테스트에서 ABC를 사용하면 API를 더욱 융통성 있고 미래의 확장에 대비할 수 있게 해준다. 11장에서 이유를 설명한다. 불행히도 파이썬 3.4 표준 라이브러리에는 slice에 대한 ABC가 없다.

In [12]:
""" [예제 10-6] 예제 10-2의 Vector 클래스에 추가된 __len__(), __getitem__() 메서드 """
# def __len__(self):
#     return len(self._components)

# def __getitem__(self, index):
#     cls = type(self) # 객체의 클래스(Vector)를 가져온다.
#     if isinstance(index, slice):
#         return cls(self._components[index])
#     elif isinstance(index, numbers.Integral): 
#         return self._components[index]
#     else:
#         msg = '{cls.__name__} indices must be integers'
#         raise TypeError(msg.format(cls=cls))

' [예제 10-6] 예제 10-2의 Vector 클래스에 추가된 __len__(), __getitem__() 메서드 '

In [13]:
v = Vector(range(7))

In [14]:
print(v7[-1])
print(v7[1:4])
print(v7[-1:])

run getitem
run numbers.Integral root in getitem
6.0
run getitem
run slice root in getitem
run len
(1.0, 2.0, 3.0)
run getitem
run slice root in getitem
run len
(6.0,)


In [15]:
v7[1, 2] # Vector는 다차원 인덱싱을 지원하지 않으므로 인덱스나 슬라이스로 구성된 튜플은 에러를 발생시킨다.

run getitem
run other root in getitem


TypeError: Vector indices must be integers

### 10.5 Vector버전 #3: 동적 속성 접근
앞에 있는 요소 몇 개는 v[0], v[1], v[2] 대신 x, y, z로 접근할 수 있다면 편리할 것이다. 
```
>>> v = Vector(range(10))
>>> v.x
0.0
>>> v.y, v.z, v.t
(1.0, 2.0, 3.0)
```
\_\_getattr\_\_( ) 특별 메서드를 이용하면 더욱 깔끔하게 구현할 수 있다. 속성을 찾지 못하면 인터프리터는 \_\_getattr\_\_( ) 메서드를 호출한다. 다시말해 my_boj.x 표현식이 주어지면, 파이썬은 my_obj 객체에 x 속성이 있는지 검사한다. 속성이 없으면 이 객체의 클래스(my_obj.\_\_class\_\_)에서 더 검색한다. 그리고 나서 상속 그래프를 따라 계속 올라간다. 그래도 x 속성을 찾지 못하면 self와 속성명을 문자열로 전달해서 my_obj의 클래스에 정의된 \_\_getattr\_\_( ) 메서드를 호출한다.

In [16]:
""" [예제 10-8] Vector 클래스에 추가된 __getattr__( ) 메서드 """
# shortcut_names = 'xyzt'

# def __getattr__(self, name):
#     cls = type(self) # Vector 클래스를 가져온다.
#     if len(name) == 1: # 한 글자 name의 위치를 찾는다. str.find()는 'yz'의 위치도 찾을 수 있으므로, 위에서 name의 길이가 1인지 확인한다.
#         pos = cls.shortcut_names.find(name)
#         if 0 <= pos < len(self._components):
#             return self._components[pos]
#     msg = '{.__name__!r} object has no attribute {!r}'
#     raise AttributeError(msg.format(cls, name)) # 실패하면 표준 메시지와 함께 AttributeError가 발생한다.

' [예제 10-8] Vector 클래스에 추가된 __getattr__( ) 메서드 '

In [17]:
v = Vector(range(10))
v.x

0.0

In [18]:
v.y, v.z, v.t

(1.0, 2.0, 3.0)

파이썬은 해당 이름의 속성을 찾지 못할 때 최후 수단으로 \_\_getatt\_\_( ) 메서드를 호출한다. 그러나 v.x = 10 문장으로 x 속성에 값을 할당할 때 v 객체에 x 속성이 추가되므로, 더 이상 v.x값을 가져오기 위해  \_\_getatt\_\_( )을 호출하지 않는다.

※ 주의: 클래스 수준에서 \_\_slots\_\_를 정의하면 새로운 객체 속성을 생성할 수 없다는 것을 알고 있으므로, 우리가 구현한 \_\_setattr\_\_( ) 대신 \_\_slots\_\_ 속성을 사용하고 싶을 것이다. 그러나 단치 객체 속성의 생성을 막기 위해 \_\_slots\_\_를 사용하는 것은 권장하지 않는다. \_\_slots\_\_는 메모리를 절약하기 위해 사용해야 한다.

In [22]:
""" [예제 10-9] 부적절한 동작. v.x에 값을 할당하더라도 에러가 발생하지는 않지만 값은 변하지 않는다. """
v = Vector(range(5))
print(v)
print(v.x)
v.x = 10
print(v.x)
print(v)

run len
(0.0, 1.0, 2.0, 3.0, 4.0)
0.0
10
run len
(0.0, 1.0, 2.0, 3.0, 4.0)


In [ ]:
# """ [예제 10-10] 예제 10-9의 문제를 개선하기 위해 __setatt__( )를 추가 """
# def __setattr__(self, name, value):
#     cls = type(self)
#     if len(name) == 1:
#         if name in cls.shorcut_names:
#             error = 'read only attribute {attr_name!r}'
#         elif name.islower():
#             error = "can't set attributes 'a' to 'z' in {cls_name!r}"
#         else:
#             error = ''
#         if error:
#             msg = error.format(cls_name=cls.__name__, attr_name=name)
#             rasise AttributeError(msg)
#     super().__setattr__(name, value) # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 함수를 호출한다.       

In [20]:
v1.x = 10

AttributeError: type object 'Vector' has no attribute 'shorcut_names'

#### 10.6 Vector버전 #4: 해싱 및 더 빠른 == 
기존 \_\_eq\_\_( ) 메서드와 함께 \_\_hash\_\_( ) 메서드를 구현하면 Vector 객체를 해시할 수 있게 된다. functools.reduce( ) 함수를 사용하면 시퀀스형을 인자로 받는 \_\_hash\_\_( ) 구현이 쉬워진다. 이 외에도 아래의 3가지 방법이 가능하다.

In [28]:
""" [예제 10-11] 0에서 5까지 정수를 XOR로 누적 계산하는 세 가지 방법 """
n = 0
for i in range(6): # 저자가 2번째로 선호
    n ^= i
print(n)

1


In [29]:
import functools
functools.reduce(lambda a, b: a^b, range(6))

1

In [30]:
import operator # 저자가 가장 선호
functools.reduce(operator.xor, range(6))

1

In [1]:
""" [예제 10-12] __hash__( ) 메서드 추가 """
from array import array
import reprlib
import math
import functools
import operator

class Vector:
    typecode = 'd'
    
    def __eq__(self, other):
        return tuple(self) == tuple(other) # 바뀐 것은 없지만 두 함수가 밀접한 관계이므로 가까이 두었다.
    
    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return fuctools.reduce(operator.xor, hashes, 0) # reduce()를 사용할 때는 세 번째 인수를 전달해서 reduce(<함수>, <반복형>, <초깃값>) 형태로 호출하고
                                                        # 'TypeError':reduce() of empty sequnce withg no initial value' 예외가 발생하지 않게 예방하는 것이 좋다.
                                                        # <초깃값>은 시퀀스가 비어 있을 때 반환되는 값이며, 리듀스 루프 안에서 첫 번째 인수로 사용된다. 따라서 함수에 대한 항등원을 
                                                        # 사용해야 한다. 예를 들어 +, |, ^ 연산에는 0이 되어야하지만, *, & 연산의 경우에는 1이 되어야 한다.

\_\_eq\_\_() 메서드를 간단히 수정해서 커다란 백터를 더 빠르고 메모리를 적게 사용하도록 바꿔볼 수 있다. 피연산자 전체를 복사해서 튜플을 만들기 때문에 요소가 두 개 밖에 없는 Vector2d의 경우에는 나쁘지 않지만, 아주 큰 다차원 벡터의 경우에는 얘기가 다르다. 
```
def __eq_(self, other):
    if len(self) != len(other): # 길이가 다르면, 다른 객체이다. 
        return False
    for a, b in zip(self, other): # 입력 중 하나가 소진되면 zip()은 경고없이 값 생산을 중단한다.
        if a != b:
            return False
    return True
```

짧게 표현하면 아래와 같다.
```
def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))
```

멋진 zip() 함수: zip() 함수는 튜플을 변수에 언패킹해서 각 변수에 병렬로 입력할 수 있다.

In [4]:
list(zip(range(3), 'ABC', [0.0 ,1.1, 2.2, 3.3])) # 반복형 중에서 어느 것이라도 소진되면 경고 메시지 없이 중단한다.

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [7]:
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0 ,1.1, 2.2, 3.3], fillvalue=-1)) # 빠진 값을 채워가면서 마지막 반복형이 소진될 때까지 튜플을 생성한다.

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

### 10.7 Vector 버전 #5: 포매팅
포멧 명시간이 언어를 확장하려면 내장 자료형에서 지원하는 포맷 코드를 다른 용도로 사용하지 않는 것이 좋다. 때문에 극좌표에서는 'p', 초평면좌표에서는 'h'를 사용한 것은 좋은 선택이다.
<li>실수형 포맷 코드: 'eEfFgGn%'</li>
<li>정수형 포맷 코드: 'bcdoxXn'</li>
<li>문자열         : 's'</li>

In [36]:
""" [예제 10-16] Vector 클래스 최종 버전에 대한 doctest와 전체코드 """
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components): # 초기화 함수가 없으면 object.__init()를 호출한다.
        self._components = array(self.typecode, components) # 백터 요소를 배열로 저장한다.
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components) # [0.0, 1.0, 2.0, 3.0, ...] 형태로 출력된다.
                                               # reprlib.repr(list(self._components))로도 가능하지만 단순한 출력을 위해 모든 항목을 list에 복사하는 것은 낭비이다.
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv) # memoryview를 바로 생성자에 전달한다. 
    
    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self) # 객체의 클래스(Vector)를 가져온다.
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral): 
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
            
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
        
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shorcut_names:
                error = 'read only attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value) # 에러가 발생하지 않을 때는 표준 동작을 위해 슈퍼클래스의 함수를 호출한다.
        
    def __eq_(self, other):
        if len(self) != len(other):
            return False
        for a, b in zip(self, other):
            if a != b:
                return False
        return True
    
    def angle(self, n): # 초구면좌표에 대한 공식 ( https://ko.wikipedia.org/wiki/초구면_좌표계 ) 을 이용해서 특정 좌표에 대한 각좌표를 계산한다.
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n==len(self) - 1) and (self[-1] < 0):
            return math.pi*2 - a
        else:
            return a
        
    def angles(self): # 요청에 따라 각 좌표를 모두 계산하는 제너레이터 표현식을 생성한다.
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'): # 초구면좌표(hyperspherical coordinate)
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords) # 좌표의 각 항목을 요청에 따라 포맷하는 제너레이터 표현식을 생성한다.
        return outer_fmt.format(', '.join(components))

In [29]:
""" 2차원 직교좌표에 대한 format() 테스트 """
v1 = Vector([3, 4])
format(v1, '.2f')

'(3.00, 4.00)'

In [28]:
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

In [31]:
""" 3차원, 7차원 직교좌표에 대한 format() 테스트 """
v3 = Vector([3, 4, 5])
format(v3)

'(3.0, 4.0, 5.0)'

In [32]:
format(Vector(range(7)))

'(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'

In [37]:
""" 2차원, 3차원, 4차원 구면좌표에 대한 format() 테스트 """
format(Vector([1, 1]), 'h') # doctest:+ELLIPSIS

'<1.4142135623730951, 0.7853981633974483>'

In [38]:
format(Vector([1, 1]), '.3eh')

'<1.414e+00, 7.854e-01>'

In [39]:
format(Vector([1, 1]), '.5fh')

'<1.41421, 0.78540>'

In [40]:
format(Vector([1, 1, 1]), 'h') # doctest:+ELLIPSIS

'<1.7320508075688772, 0.9553166181245093, 0.7853981633974483>'

In [41]:
format(Vector([2, 2, 2]), '.3fh')

'<3.464, 0.955, 0.785>'

In [43]:
format(Vector([0, 0, 0]), '.5fh') 

'<0.00000, 0.00000, 0.00000>'

In [44]:
format(Vector([-1, -1, -1, -1]), 'h') # doctest:+ELLIPSIS

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'